In [6]:
import sqlite3
import pandas as pd
import numpy as np
import math
from typing import Dict

In [8]:
db_file = "rutgers_buses.db"

with sqlite3.connect(db_file) as con:
    tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", con)
    print(tables)
    
    df = pd.read_sql_query("SELECT * FROM Bus_Logs", con)
    names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", con)['name'].tolist()
    dfs = {name: pd.read_sql_query(f"SELECT * FROM \"{name}\";", con) for name in names}
print(df.shape)

              name
0          Systems
1           Routes
2            Buses
3            Stops
4      Route_Stops
5         ETA_Logs
6         Bus_Logs
7  sqlite_sequence
(1696450, 8)


Haversine distance formula is used to find speed later on. I found this formula online.

In [7]:
def distance(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    deltaphi = np.radians(lat2 - lat1)
    deltalambda = np.radians(lon2 - lon1)
    a = np.sin(deltaphi/2.0)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(deltalambda/2.0)**2
    return R * 2 * np.arcsin(np.sqrt(a))

This cell will do feature engineering and calculate the actual arrival time as well as the instantaneous speed vs the 1 min average speed.

In [ ]:
DB_FILE = "rutgers_buses.db"
BUS_TBL = "Bus_Logs"
ETA_TBL = "ETA_Logs"

BUS_LOG_ID = "log_id"
BUS_TS = "timestamp"
BUS_BUS_ID = "bus_id"
BUS_LAT = "latitude"
BUS_LON = "longitude"
BUS_PAX = "pax_load"
BUS_ARRIVED_STOP = "arrived_stop_id"

ETA_LOG_ID = "log_id"
ETA_STOP = "stop_id"
ETA_SECONDS = "eta_seconds"
ETA_SORT = "sort_order"

MAX_TIME_BEFORE_STOP = 60 * 60 
CHUNKSIZE = 200000

with sqlite3.connect(DB_FILE) as con:
    bus = pd.read_sql_query(f"SELECT * FROM {BUS_TBL}", con)
    eta_cols = pd.read_sql_query(f"PRAGMA table_info('{ETA_TBL}')", con)['name'].tolist()

required_bus_cols = {BUS_LOG_ID, BUS_TS, BUS_BUS_ID, BUS_LAT, BUS_LON, BUS_PAX, BUS_ARRIVED_STOP}
missing = required_bus_cols - set(bus.columns)


route_col_in_bus = next((c for c in bus.columns if "route" in c.lower() and "id" in c.lower()), None)

bus = bus.copy()
timestamp_values = pd.to_numeric(bus[BUS_TS], errors="coerce").dropna()
unit = "s"

bus[BUS_LAT] = pd.to_numeric(bus[BUS_LAT], errors="coerce")
bus[BUS_LON] = pd.to_numeric(bus[BUS_LON], errors="coerce")
bus[BUS_PAX] = pd.to_numeric(bus[BUS_PAX], errors="coerce")

bus_by_log = bus.set_index(BUS_LOG_ID, drop=False)

bus_stop_events = bus.loc[bus[BUS_ARRIVED_STOP].notna(), [BUS_BUS_ID, "timestamp", BUS_ARRIVED_STOP, BUS_LAT, BUS_LON]].copy()
bus_stop_events = bus_stop_events.rename(columns={BUS_ARRIVED_STOP: "event_stop_id", "timestamp": "event_ts"})

bus_sorted = bus.sort_values([BUS_BUS_ID, "timestamp"]).reset_index(drop=True)
segment_map: Dict[str, Dict[str, np.ndarray]] = {}
for bus_id, grp in bus_sorted.groupby(BUS_BUS_ID):
    seq = grp.sort_values("timestamp")
    ts = seq["timestamp"].values.astype("datetime64[ns]")
    lat = seq[BUS_LAT].values if BUS_LAT in seq.columns else None
    lon = seq[BUS_LON].values if BUS_LON in seq.columns else None
    if len(ts) < 2 or lat is None or lon is None:
        segment_map[bus_id] = {"ts": np.array([], dtype="datetime64[ns]"),
                         "speed": np.array([], dtype=float),
                         "dt": np.array([], dtype=float)}
        continue
    dt = (ts[1:] - ts[:-1]) / np.timedelta64(1, "s")
    dist = distance(lat[:-1].astype(float), lon[:-1].astype(float),
                                lat[1:].astype(float), lon[1:].astype(float))
    speed = dist / dt
    segment_map[bus_id] = {"ts": ts[1:], "speed": speed, "dt": dt}

def calculate_speed_per_row(bus_id, start_timestamp):
    if bus_id not in segment_map:
        return np.nan, np.nan
    rec = segment_map[bus_id]
    seg_ts = rec["ts"]
    if seg_ts.size == 0:
        return np.nan, np.nan
    start = np.datetime64(pd.to_datetime(start_timestamp).to_datetime64())
    index = np.searchsorted(seg_ts, start) - 1
    speed_prev = np.nan
    if index >= 0:
        if rec["dt"][index] > 0 and rec["dt"][index] <= 30:
            speed_prev = float(rec["speed"][index])
    lower = start - np.timedelta64(60, "s")
    mask = (seg_ts > lower) & (seg_ts <= start)
    if mask.any():
        speeds = rec["speed"][mask]
        speeds = speeds[np.isfinite(speeds)]
        speed_1min = float(np.nanmean(speeds)) if speeds.size else np.nan
    else:
        speed_1min = np.nan
    return speed_prev, speed_1min

route_next_map = {}
try:
    with sqlite3.connect(DB_FILE) as _con:
        rs = pd.read_sql_query("SELECT * FROM Route_Stops", _con)
except Exception:
    try:
        with sqlite3.connect(DB_FILE) as _con:
            rs = pd.read_sql_query("SELECT * FROM route_stops", _con)
    except Exception:
        rs = pd.DataFrame(columns=["route_id", "stop_id", "position_on_route"])

rs_route_col = next((c for c in rs.columns if c.lower() in ("route_id_from_stop","route_myid","route_id","route")), None)
rs_stop_col = next((c for c in rs.columns if c.lower() in ("stop_id","stop_myid","stop")), None)
rs_pos_col  = next((c for c in rs.columns if c.lower() in ("position_on_route","position","order")), None)

if not rs.empty and rs_route_col and rs_stop_col and rs_pos_col:
    rs[rs_route_col] = rs[rs_route_col].astype(str)
    rs[rs_stop_col] = rs[rs_stop_col].astype(str)
    rs = rs.sort_values([rs_route_col, rs_pos_col]).reset_index(drop=True)
    for rid, g in rs.groupby(rs_route_col):
        g2 = g.sort_values(rs_pos_col).reset_index(drop=True)
        stops = g2[rs_stop_col].astype(str).tolist()
        if not stops:
            continue
        next = {}
        L = len(stops)
        if L == 1:
            next[stops[0]] = stops[0]
        else:
            for i in range(L):
                if i == L - 1:
                    next[stops[i]] = stops[1] 
                else:
                   next[stops[i]] = stops[i + 1]  
        route_next_map[str(rid)] = {"stops": stops, "next": next, "max_pos": len(stops) - 1}
def find_next_stop_id(route_id, stop_id):
    if route_id is None or pd.isna(route_id) or stop_id is None or pd.isna(stop_id):
        return None
    entry = route_next_map.get(str(route_id))
    if not entry:
        return None
    return entry["next"].get(str(stop_id))

features_parts = []
with sqlite3.connect(DB_FILE) as con:
    select_cols = [f"{ETA_LOG_ID} AS log_id", f"{ETA_STOP} AS stop_id",
                   f"{ETA_SECONDS} AS eta_seconds", f"{ETA_SORT} AS sort_order"]
    sql = f"SELECT {', '.join(select_cols)} FROM {ETA_TBL}"
    total_eta = pd.read_sql_query(f"SELECT COUNT(*) AS c FROM {ETA_TBL}", con).iloc[0, 0]
    total_chunks = max(1, math.ceil(total_eta / CHUNKSIZE))
    chunk_iter = pd.read_sql_query(sql, con, chunksize=CHUNKSIZE)

    base_idx = 0
    processed = 0
    chunk_idx = 0
    for chunk in chunk_iter:
        chunk_idx += 1
        chunk = chunk.reset_index(drop=True)
        chunk["eta_row_id"] = np.arange(len(chunk)) + base_idx
        base_idx += len(chunk)
        print(f"Chunk {chunk_idx}/{total_chunks} — rows {processed + 1} / {total_eta}", flush=True)
        cols = [BUS_LOG_ID, BUS_BUS_ID, "timestamp", BUS_LAT, BUS_LON, BUS_PAX, BUS_ARRIVED_STOP]
        if route_col_in_bus and route_col_in_bus in bus_by_log.columns:
            cols.append(route_col_in_bus)
        bus_by_log_view = bus_by_log.reset_index(drop=True)[cols]

        merged = chunk.merge(
            bus_by_log_view,
            left_on="log_id",
            right_on=BUS_LOG_ID,
            how="left",
            suffixes=("_eta", "_bus")
        ).rename(columns={BUS_BUS_ID: "bus_id", "timestamp": "start_ts", BUS_PAX: "bus_pax"})

        if route_col_in_bus and route_col_in_bus in merged.columns:
            merged = merged.rename(columns={route_col_in_bus: "route_id"})

        merged_valid = merged[merged["start_ts"].notna()].copy()
        if BUS_ARRIVED_STOP in merged_valid.columns:
            merged_valid = merged_valid[ merged_valid[BUS_ARRIVED_STOP].isna() | (merged_valid[BUS_ARRIVED_STOP] != merged_valid["stop_id"]) ].copy()
        if merged_valid.empty:
            processed += len(chunk)
            continue

        bus_events_map = {}
        for bus_id, g in bus_stop_events.groupby(BUS_BUS_ID):
            g2 = g.sort_values("event_ts")[["event_ts", "event_stop_id"]].dropna(subset=["event_ts"]).reset_index(drop=True)
            if g2.empty:
                continue
            bus_events_map[bus_id] = {
                "ts": g2["event_ts"].values.astype("datetime64[ns]"),
                "stop": g2["event_stop_id"].astype(object).values
            }

        def lookup_arrival_ts(bid, stop_id, start_ts, route_id=None):
            if bid not in bus_events_map or pd.isna(stop_id) or pd.isna(start_ts):
                return pd.NaT
            arr = bus_events_map[bid]
            try:
                st64 = np.datetime64(pd.to_datetime(start_ts).to_datetime64())
            except Exception:
                return pd.NaT
            index = np.searchsorted(arr["ts"], st64)
            next_stop = find_next_stop_id(route_id, stop_id) if route_id is not None else None
            target_stop_str = str(stop_id)
            while index < arr["ts"].size:
                event_stop = str(int(arr["stop"][index]))
                delta_s = (arr["ts"][index] - st64) / np.timedelta64(1, "s")
                if next_stop is not None and event_stop == str(next_stop):
                    return pd.NaT
                if event_stop == target_stop_str:
                    if 0 < delta_s <= MAX_TIME_BEFORE_STOP:
                        return pd.to_datetime(arr["ts"][index])
                    return pd.NaT
                if delta_s > MAX_TIME_BEFORE_STOP:
                    return pd.NaT
                index += 1
            return pd.NaT
        merged_valid["actual_arrival_ts"] = merged_valid.apply(
            lambda r: lookup_arrival_ts(r["bus_id"], r["stop_id"], r["start_ts"], route_id=r.get("route_id")),
            axis=1
        )
        merged_valid["pred_eta_s"] = pd.to_numeric(merged_valid["eta_seconds"], errors="coerce")
        merged_valid["actual_travel_s"] = (pd.to_datetime(merged_valid["actual_arrival_ts"]) - pd.to_datetime(merged_valid["start_ts"])).dt.total_seconds()
        merged_valid["eta_error_s"] = np.where(merged_valid["actual_travel_s"].notna() & merged_valid["pred_eta_s"].notna(),
                                               merged_valid["actual_travel_s"] - merged_valid["pred_eta_s"],
                                               np.nan)
        merged_valid["pax_load"] = pd.to_numeric(merged_valid.get("eta_pax"), errors="coerce")
        merged_valid.loc[merged_valid["pax_load"].isna(), "pax_load"] = merged_valid.loc[merged_valid["pax_load"].isna(), "bus_pax"]
        merged_valid["start_ts"] = pd.to_datetime(merged_valid["start_ts"])
        merged_valid["hour"] = merged_valid["start_ts"].dt.hour
        merged_valid["time_of_day_s"] = merged_valid["hour"] * 3600 + merged_valid["start_ts"].dt.minute * 60 + merged_valid["start_ts"].dt.second

        sp_prev_list = []
        sp_1min_list = []
        for _, row in merged_valid.iterrows():
            bus_id = row["bus_id"]
            st = row["start_ts"]
            sp_prev, sp_1min = calculate_speed_per_row(bus_id, st)
            sp_prev_list.append(sp_prev)
            sp_1min_list.append(sp_1min)
        merged_valid["speed_prev_mps"] = sp_prev_list
        merged_valid["speed_1min_mps"] = sp_1min_list

        out = merged_valid[[
            "eta_row_id", "log_id", "bus_id", "stop_id", "sort_order", "eta_seconds", "pred_eta_s",
            "actual_arrival_ts", "actual_travel_s", "eta_error_s",
            "pax_load", "hour", "time_of_day_s", "speed_prev_mps", "speed_1min_mps"
        ]].copy()
        features_parts.append(out)
        processed += len(chunk)
        print(f"[ETA] completed {processed} / {total_eta} rows ({chunk_idx}/{total_chunks} chunks)", flush=True)
if features_parts:
    features = pd.concat(features_parts, ignore_index=True)
else:
    features = pd.DataFrame(columns=["eta_row_id","log_id","bus_id","stop_id","sort_order","eta_seconds","pred_eta_s",
                                     "actual_arrival_ts","actual_travel_s","eta_error_s","pax_load","hour",
                                     "time_of_day_s","speed_prev_mps","speed_1min_mps"])

features = features.sort_values(["bus_id","eta_row_id"]).reset_index(drop=True)
print("Prepared features rows:", len(features))

/var/folders/j7/zg72qj2521b4v_sw10nk4ntm0000gn/T/ipykernel_14877/1806519234.py:59: RuntimeWarning: divide by zero encountered in divide
  speed = dist / dt
/var/folders/j7/zg72qj2521b4v_sw10nk4ntm0000gn/T/ipykernel_14877/1806519234.py:59: RuntimeWarning: invalid value encountered in divide
  speed = dist / dt


route: 26280 sample next mappings: [('10035', '27767'), ('27767', '10038'), ('10038', '10035'), ('10034', '10041'), ('10041', '10052'), ('10052', '10071'), ('10071', '10029'), ('10029', '10065'), ('10065', '12913'), ('12913', '10037'), ('10037', '10059'), ('10059', '10042'), ('10042', '10061'), ('10061', '10036'), ('10036', '62662'), ('62662', '10038')]
route: 26281 sample next mappings: [('10035', '27767'), ('27767', '10075'), ('10075', '10037'), ('10037', '10059'), ('10059', '10042'), ('10042', '10026'), ('10026', '10061'), ('10061', '10036'), ('10036', '10038'), ('10038', '10071'), ('10071', '10029'), ('10029', '10065'), ('10065', '10052'), ('10052', '10039'), ('10039', '21050'), ('21050', '27767')]
route: 26435 sample next mappings: [('10035', '27767'), ('27767', '10038'), ('10038', '10035'), ('10034', '10041'), ('10041', '10052'), ('10052', '10071'), ('10071', '10029'), ('10029', '12913'), ('12913', '10037'), ('10037', '10059'), ('10059', '10042'), ('10042', '10026'), ('10026', '1

KeyboardInterrupt: 

In [ ]:
#DRops rows where any data is missing
features_clean = features.dropna(how="any").reset_index(drop=True)
print(f"features: {len(features)} rows -> features_clean: {len(features_clean)} rows")
print(features_clean.head())
features_clean.to_csv("features_clean_updated_2.csv", index=False)

features: 11017166 rows -> features_clean: 7178015 rows
   eta_row_id  log_id  bus_id  stop_id  sort_order  eta_seconds  pred_eta_s  \
0      293333   23284    4850    10039           0           92          92   
1      293335   23284    4850    10060           2          497         497   
2      293336   23284    4850    10035           3          812         812   
3      293337   23284    4850    27767           4         1000        1000   
4      293338   23284    4850    10038           5         1159        1159   

    actual_arrival_ts  actual_travel_s  eta_error_s  pax_load  hour  \
0 2025-11-10 12:21:23            167.0         75.0       4.0    12   
1 2025-11-10 12:25:14            398.0        -99.0       4.0    12   
2 2025-11-10 12:30:45            729.0        -83.0       4.0    12   
3 2025-11-10 12:35:59           1043.0         43.0       4.0    12   
4 2025-11-10 12:38:20           1184.0         25.0       4.0    12   

   time_of_day_s  speed_prev_mps  speed_1m

In [ ]:
with sqlite3.connect(DB_FILE) as con:
    routes = pd.read_sql_query("SELECT route_myid, short_name FROM Routes", con)
    lx_route_myids = routes[routes["short_name"].astype(str).str.upper() == "LX"]["route_myid"].unique()
    bus_route_map = pd.read_sql_query("SELECT log_id, route_myid FROM Bus_Logs", con)
lx_log_ids = bus_route_map[bus_route_map["route_myid"].isin(lx_route_myids)]["log_id"].unique()
lx_features = features_clean[features_clean["log_id"].isin(lx_log_ids)].copy()
lx_features.to_csv("lx_features_updated_2.csv", index=False)

We clean the LX dataset so that it drops every row that contains at least one NA feature

In [ ]:
with sqlite3.connect(DB_FILE) as con:
    routes = pd.read_sql_query("SELECT route_myid, short_name FROM Routes", con)
    b_route_myids = routes[routes["short_name"].astype(str).str.upper() == "B"]["route_myid"].unique()
    bus_route_map = pd.read_sql_query("SELECT log_id, route_myid FROM Bus_Logs", con)
b_log_ids = bus_route_map[bus_route_map["route_myid"].isin(b_route_myids)]["log_id"].unique()
b_features = features_clean[features_clean["log_id"].isin(b_log_ids)].copy()
b_features.to_csv("b_features_2.csv", index=False)